## Loading Pretrained Word Embeddings

In the previous section we trained custom word embeddings. However, we can also use pretrained word embeddings. 
<a href="https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras/"> Tuto Link</a>

Several types of pretrained word embeddings exist, however we will be using the GloVe word embeddings from Stanford NLP since it is the most famous one and commonly used. The word embeddings can be downloaded from <a href="https://nlp.stanford.edu/projects/glove/">this link.</a>

The smallest file is named "Glove.6B.zip". The size of the file is 822 MB. The file contains 50, 100, 200, and 300 dimensional word vectors for 400k words. We will be using the 100 dimensional vector.

The process is quite similar. First we have to import the required libraries:

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding

Using TensorFlow backend.


Next, we have to create our corpus followed by the labels.

In [2]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In the last section, we used one_hot function to convert text to vectors. Another approach is to use Tokenizer function from keras.preprocessing.text library.

You simply have to pass your corpus to the Tokenizer's fit_on_text method.

In [3]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

To get the number of unique words in the text, you can simply count the length of word_index dictionary of the word_tokenizer object. Remember to add 1 with the vocabulary size. This is to store the dimensions for the words for which no pretrained word embeddings exist.

In [4]:
vocab_length = len(word_tokenizer.word_index) + 1

Finally, to convert sentences to their numeric counterpart, call the texts_to_sequences function and pass it the whole corpus.

In [5]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)

[[14, 3, 15, 16, 1], [4, 17, 6, 9, 5, 7, 2], [18, 19, 20, 2, 3, 21], [22, 23], [24, 25, 26, 27, 5, 7, 2], [28, 8, 9, 29], [30, 31, 32, 8, 33, 1], [2, 3, 8, 34, 1], [10, 11], [35, 36, 37], [12, 38], [2, 6, 39, 40], [5, 41, 13, 7, 4, 1], [4, 1, 6, 10], [5, 42, 13, 43], [4, 11, 3, 12]]


The next step is to find the number of words in the longest sentence and then to apply padding to the sentences having shorter lengths than the length of the longest sentence.

In [6]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[14  3 15 16  1  0  0]
 [ 4 17  6  9  5  7  2]
 [18 19 20  2  3 21  0]
 [22 23  0  0  0  0  0]
 [24 25 26 27  5  7  2]
 [28  8  9 29  0  0  0]
 [30 31 32  8 33  1  0]
 [ 2  3  8 34  1  0  0]
 [10 11  0  0  0  0  0]
 [35 36 37  0  0  0  0]
 [12 38  0  0  0  0  0]
 [ 2  6 39 40  0  0  0]
 [ 5 41 13  7  4  1  0]
 [ 4  1  6 10  0  0  0]
 [ 5 42 13 43  0  0  0]
 [ 4 11  3 12  0  0  0]]


We have converted our sentences into padded sequence of numbers. The next step is to load the GloVe word embeddings and then create our embedding matrix that contains the words in our corpus and their corresponding values from GloVe embeddings. Run the following script:

In [7]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('./data/archive/glove.6B.100d.txt', encoding="utf8")

In the script above, in addition to loading the GloVe embeddings, we also imported a few libraries. We will see the use of these libraries in the upcoming section. Here notice that we loaded glove.6B.100d.txt file. This file contains 100 dimensional word embeddings. We also created an empty dictionary that will store our word embeddings.

If you open the file, you will see a word at the beginning of each line followed by set of 100 numbers. The numbers form the 100 dimensional vector for the word at the begining of each line.

We will create a dictionary that will contain words as keys and the corresponding 100 dimensional vectors as values, in the form of an array. Execute the following script:

In [8]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype="float32")
    embeddings_dictionary[word] = vector_dimensions
    
glove_file.close()

The dictionary embeddings_dictionary now contains words and corresponding GloVe embeddings for all the words.

The dictionary embeddings_dictionary now contains words and corresponding GloVe embeddings for all the words.

We want the word embeddings for only those words that are present in our corpus. We will create a two dimensional numpy array of 44 (size of vocabulary) rows and 100 columns. The array will initially contain zeros. The array will be named as embedding_matrix

Next, we will iterate through each word in our corpus by traversing the word_tokenizer.word_index dictionary that contains our words and their corresponding index.

Each word will be passed as key to the embedding_dictionary to retrieve the corresponding 100 dimensional vector for the word. The 100 dimensional vector will then be stored at the corresponding index of the word in the embedding_matrix. Look at the following script:

In [9]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

Our embedding_matrix now contains pretrained word embeddings for the words in our corpus.

Now we are ready to create our sequential model. Look at the following script:

In [10]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

The script remains the same, except for the embedding layer. Here in the embedding layer, the first parameter is the size of the vacabulary. The second parameter is the vector dimension of the output vector. Since we are using pretrained word embeddings that contain 100 dimensional vector, we set the vector dimension to 100.

Another very important attribute of the Embedding() layer that we did not use in the last section is weights. You can pass your pretrained embedding matrix as default weights to the weights parameter. And since we are not training the embedding layer, the trainable attribute has been set to False.

Let's compile our model and see the summary of our model:

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 100)            4400      
_________________________________________________________________
flatten (Flatten)            (None, 700)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 701       
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


We are again using adam as the optizer to minimize the loss. The loss function being used is binary_crossentropy. And we want to see the results in the form of accuracy so acc has been passed as the value for the metrics attribute.

You can see that since we have 44 words in our vocabulary and each word will be represented as a 100 dimensional vector, the number of parameters for the embedding layer will be 44 x 100 = 4400. The output from the embedding layer will be a 2D vector with 7 rows (1 for each word in the sentence) and 100 columns. The output from the embedding layer will be flattened so that it can be used with the dense layer. Finally the dense layer is used to make predictions.

Execute the following script to train the algorithms:

In [12]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6514 - acc: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 664us/step - loss: 0.6252 - acc: 0.5625
Epoch 3/100
1/1 [==============================] - 0s 816us/step - loss: 0.6014 - acc: 0.6250
Epoch 4/100
1/1 [==============================] - 0s 843us/step - loss: 0.5797 - acc: 0.6875
Epoch 5/100
1/1 [==============================] - 0s 767us/step - loss: 0.5598 - acc: 0.8125
Epoch 6/100
1/1 [==============================] - 0s 739us/step - loss: 0.5412 - acc: 0.8750
Epoch 7/100
1/1 [==============================] - 0s 688us/step - loss: 0.5239 - acc: 0.8750
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5074 - acc: 0.9375
Epoch 9/100
1/1 [==============================] - 0s 760us/step - loss: 0.4916 - acc: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4763 - acc: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 813us/step

Once the algorithm is trained, run the following script to evaluate the peformance of the algorithm.

In [13]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


## Word Embeddings with Keras Functional API
In the last section, we saw how word embeddings can be used with the Keras sequential API. While the sequential API is a good starting point for beginners, as it allows you to quickly create deep learning models, it is extremely important to know how Keras Functional API works. Most of the advanced deep learning models involving multiple inputs and outputs use the Functional API.

In this section, we will see how we can implement embedding layer with Keras Functional API.

The rest of the script remains similar as it was in the last section. The only change will be in the development of a deep learning model. Let's implement the same deep learning model as we implemented in the last section with Keras Functional API.

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

deep_inputs = Input(shape=(length_long_sentence,))
embedding = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)(deep_inputs) # line A
flatten = Flatten()(embedding)
hidden = Dense(1, activation='sigmoid')(flatten)
model = Model(inputs=deep_inputs, outputs=hidden)

In the Keras Functional API, you have to define the input layer separately before the embedding layer. In the input, layer you have to simply pass the length of input vector. To specify that previous layer as input to the next layer, the previous layer is passed as a parameter inside the parenthesis, at the end of the next layer.

For instance, in the above script, you can see that deep_inputs is passed as parameter at the end of the embedding layer. Similarly, embedding is passed as input at the end of the Flatten() layer and so on.

Finally, in the Model(), you have to pass the input layer, and the final output layer.

Let's now compile the model and take a look at the summary of the model.

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 7, 100)            4400      
_________________________________________________________________
flatten_1 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 701       
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


In the model summary, you can see the input layer as a separate layer before the embedding layer. The rest of the model remains the same.

Finally, the process to fit and evaluate the model is same as the one used in Sequential API:

In [20]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 912us/step - loss: 0.0717 - acc: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 818us/step - loss: 0.0708 - acc: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0699 - acc: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 835us/step - loss: 0.0690 - acc: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 863us/step - loss: 0.0681 - acc: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 713us/step - loss: 0.0673 - acc: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 632us/step - loss: 0.0664 - acc: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0656 - acc: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 839us/step - loss: 0.0648 - acc: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 895us/step - loss: 0.0640 - acc: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 910us/st

In [21]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)

print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


## Conclusion
To use text data as input to the deep learning model, we need to convert text to numbers. However unlike machine learning models, passing sparse vector of huge sizes can greately affect deep learning models. Therefore, we need to convert our text to small dense vectors. Word embeddings help us convert text to dense vectors.

In this article we saw how word embeddings can be implemented with Keras deep learning library. We implemented the custom word embeddings as well as used pretrained word embedddings to solve simple classification task. Finally, we also saw how to implement word embeddings with Keras Functional API.